In [1]:
import sys
sys.path.append("..")

from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

from langchain.document_loaders.pdf import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.prompts import ChatPromptTemplate
from langchain_community.llms.ollama import Ollama

import warnings
warnings.filterwarnings("ignore")

In [2]:
DATA_PATH = "/Users/anuragkotiyal/Desktop/engaige/docs"

In [3]:
def load_docs(data_dir):
    
    """ A simple function that loads data from all pdfs inside a directory.

    Args:
        DATA_DIR (path): Path to the directory where pdf documents are stored.

    Returns:
        list(tuple): returns a list of tuples where each tuple contains "page content" of the document and
        some meta deta like "source" and "page number" i.e., [Document(page_content = "Tax information ...",
        metadata = {"source": "doc_1.pdf", "page": 10})].
    """
    
    # create an instance of document loader using the pypdfdirectoryloader from langchain
    document_loader = PyPDFDirectoryLoader(data_dir)
    
    return document_loader.load()

def split_docs_into_chunks(documents, chunk_size = 500, chunk_overlap = 50):
    
    """ A function that split a document into chunks of specific sizes like 500 characters.

    Args:
        documents (list(tuple)): a list of tuples where each tuple contains "page content" of the document and
        some meta deta like "source" and "page number".

    Returns:
        list(tuples): similar to the format in which data came but any document larger than chunk_size has been split
        into multiple chunks.
    """
    
    # create an instance of recursive character text splitter from langchain
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = chunk_size,
        chunk_overlap = chunk_overlap,
        length_function = len,
        is_separator_regex = False,
    )
    
    return text_splitter.split_documents(documents)    

def get_chunk_ids(chunks):

    """ A function that creates a unique identifier for each chunk in the database like "docs/doc_name:page_num:chunk_id".
    A chunk id of "docs/doc_1.pdf:10:5" refers to the 5th chunk on page 10 of doc_1.pdf. 

    Returns:
        list(tuple): returns a list of tuples where each tuple contains "page content" of the document and
        some meta deta like "source", "page number", and "chunk_id"
    """

    last_page_id = None
    current_chunk_index = 0

    for chunk in chunks:
        # get source and page number from metadata to create a current page id
        source = chunk.metadata.get("source")
        page = chunk.metadata.get("page")
        current_page_id = f"{source}:{page}"

        # if the page ID is the same as the last one, increment the index.
        if current_page_id == last_page_id:
            current_chunk_index += 1
        else:
            current_chunk_index = 0

        # calculate the chunk ID.
        chunk_id = f"{current_page_id}:{current_chunk_index}"
        last_page_id = current_page_id

        # add it to the page meta-data.
        chunk.metadata["id"] = chunk_id

    return chunks

In [4]:
documents = load_docs(DATA_PATH)

In [5]:
documents[0:10]

[Document(page_content='Private Vorsorge \nInformationen vor Vertragsabschluss Basispaket\nParkAllee ', metadata={'source': '/Users/anuragkotiyal/Desktop/engaige/docs/ParkAllee.pdf', 'page': 0}),
 Document(page_content='PA/D/100 6/III/11/21Gesetzlich vorgeschriebene Informationen \nfür Ihren Versicherungsvertrag\nFragen und Antworten rund um Standard Life\nSteuerinformationen\nSteuerliche Behandlung Ihrer ParkAllee  \nf\nondsgebundene Renten versicherung\nDas Kleingedruckte mal ganz groß \nVersicherungsbedingungen\n für Ihre ParkAllee  \nfondsgebundene R entenversicherung', metadata={'source': '/Users/anuragkotiyal/Desktop/engaige/docs/ParkAllee.pdf', 'page': 1}),
 Document(page_content='  \nPA/D/1006/III/11/21   \n       \n \n \nInhaltsübersicht  \nGesetzlich vorgeschriebene Informationen für Ihren Versicherungsvertrag  \nSteuerinformationen  \nDas Kleingedruckte – mal ganz groß: Allgemeine Versicherungsbedingungen ', metadata={'source': '/Users/anuragkotiyal/Desktop/engaige/docs/Park

In [6]:
chunks = split_docs_into_chunks(documents)
chunks_with_ids = get_chunk_ids(chunks)

In [7]:
chunks_with_ids[0:10]

[Document(page_content='Private Vorsorge \nInformationen vor Vertragsabschluss Basispaket\nParkAllee', metadata={'source': '/Users/anuragkotiyal/Desktop/engaige/docs/ParkAllee.pdf', 'page': 0, 'id': '/Users/anuragkotiyal/Desktop/engaige/docs/ParkAllee.pdf:0:0'}),
 Document(page_content='PA/D/100 6/III/11/21Gesetzlich vorgeschriebene Informationen \nfür Ihren Versicherungsvertrag\nFragen und Antworten rund um Standard Life\nSteuerinformationen\nSteuerliche Behandlung Ihrer ParkAllee  \nf\nondsgebundene Renten versicherung\nDas Kleingedruckte mal ganz groß \nVersicherungsbedingungen\n für Ihre ParkAllee  \nfondsgebundene R entenversicherung', metadata={'source': '/Users/anuragkotiyal/Desktop/engaige/docs/ParkAllee.pdf', 'page': 1, 'id': '/Users/anuragkotiyal/Desktop/engaige/docs/ParkAllee.pdf:1:0'}),
 Document(page_content='PA/D/1006/III/11/21   \n       \n \n \nInhaltsübersicht  \nGesetzlich vorgeschriebene Informationen für Ihren Versicherungsvertrag  \nSteuerinformationen  \nDas Klein

In [8]:
print(f"The number of loaded docs were {len(documents)} and they were split into {len(chunks)} chunks.")

The number of loaded docs were 83 and they were split into 546 chunks.


In [9]:
# pre-trained hugging face embedding model used to embed user query and loaded data from pdfs
model_name = "danielheinz/e5-base-sts-en-de"

# create a dictionary with model configuration options, specifying to use the CPU for computations
model_kwargs = {'device':'cpu'}

# create a dictionary with encoding options, specifically setting 'normalize_embeddings' to False
encode_kwargs = {'normalize_embeddings': False}

# initialize an instance of HuggingFaceEmbeddings with the specified parameters
embeddings = HuggingFaceEmbeddings(
    model_name = model_name,     
    model_kwargs = model_kwargs, 
    encode_kwargs = encode_kwargs 
)

In [10]:
text = "Dies ist ein Testdokument."
query_result = embeddings.embed_query(text)
query_result[:3]

[-0.0462304949760437, -0.06915473937988281, 0.013720380142331123]

In [11]:
print(f"This embedding model creates vector embeddings for chunks in {len(query_result)} dimensions.")

This embedding model creates vector embeddings for chunks in 768 dimensions.


In [12]:
db = FAISS.from_documents(chunks_with_ids, embeddings)

In [13]:
question = "Welche Arten von Fonds?"
searchDocs = db.similarity_search_with_score(question, k = 3)
for doc, _ in searchDocs:
    print(doc.page_content)
    print(f"Chunk ID - {doc.metadata['id']}")
    print("-----")

Ihr Mindestanteil an jedem einzelnen von Ihnen ausgewählten Fonds beträgt 1 Prozent.  
5.5 Welche Arten von Fonds bieten wir an?  
Die von Ihnen gewählten Fonds ordnen wir dem Fondsvermögen in Ihrem Vertrag zu. Sie können aus drei Fondsarten auswählen, St andard Life Fonds, Managed Portfolios sowie Publikumsfonds verschiedener 
Fondsgesellschaften.
Chunk ID - /Users/anuragkotiyal/Desktop/engaige/docs/ParkAllee.pdf:19:7
-----
5 Ihr Fondsvermögen .............................................................................................................................4
5.1 Was ist Ihr Fondsvermögen?.................................................................................................................4
5.2 Wie berechnen wir Ihr Fondsvermögen?...............................................................................................4
Chunk ID - /Users/anuragkotiyal/Desktop/engaige/docs/Basispaket+WeitBlick.pdf:10:5
-----
5.3 Was gilt unter außergewöhnlichen Umständen für di

In [14]:
PROMPT_TEMPLATE = """
Beantworten Sie die Frage nur basierend auf dem folgenden Kontext:

{context}

---

Beantworten Sie die Frage anhand des obigen Kontexts: {question}
"""

In [15]:
query_text = "Welche Arten von Fonds werden angeboten?"

searchDocs = db.similarity_search_with_score(question, k = 3)
context = " ".join([doc.page_content for doc, _ in searchDocs])

prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
prompt = prompt_template.format(context = context, question = query_text)

In [16]:
print(prompt)

Human: 
Beantworten Sie die Frage nur basierend auf dem folgenden Kontext:

Ihr Mindestanteil an jedem einzelnen von Ihnen ausgewählten Fonds beträgt 1 Prozent.  
5.5 Welche Arten von Fonds bieten wir an?  
Die von Ihnen gewählten Fonds ordnen wir dem Fondsvermögen in Ihrem Vertrag zu. Sie können aus drei Fondsarten auswählen, St andard Life Fonds, Managed Portfolios sowie Publikumsfonds verschiedener 
Fondsgesellschaften. 5 Ihr Fondsvermögen .............................................................................................................................4
5.1 Was ist Ihr Fondsvermögen?.................................................................................................................4
5.2 Wie berechnen wir Ihr Fondsvermögen?...............................................................................................4 5.3 Was gilt unter außergewöhnlichen Umständen für die Berechnung von Vermögenswerten? ..............4
5.4 Wie investieren wir Ihren Einmalbeitr

In [17]:
# initialise Ollama minstral model -> need to run ollama serve from terminal before using Ollama
model = Ollama(model = "llama3")

In [18]:
def get_answer(question: str):
    
    """ A function that retrives similar documents from our faiss database and pass an enhanced query
    through Ollama minstral model to receive a coherent and concide answer to the base query.

    Args:
        question (str): a question asked by the user.

    Returns:
        str: formatted text response from the minstral model that contains source material for the answer.
    """
    
    # retrieve top 3 relevant document chunks from the database based on user's query 
    results = db.similarity_search_with_score(question, k = 3)
    context = " ".join([doc.page_content for doc, _ in results])
    
    # create a prompt template for Ollama
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    
    # create a proper prompt using langchains LLM prompt
    prompt = prompt_template.format(context = context, question = question)
    
    # get sources of the relevant docs in this case the unique id we created for the chunks
    sources = [doc.metadata.get("id", None) for doc, _score in results]
    
    # get a response to the enhanced query from Ollama
    response_text = model.invoke(prompt)
    
    # format response so that it contains model's answer as well the source documents it used to generate that answer
    formatted_response = f"Response: {response_text}\n\nSources: {sources}"
    
    return formatted_response

In [19]:
# question 1 in German from query.txt
query = "Welche Arten von Fonds werden angeboten?"
answer = get_answer(question = query)

In [20]:
print(answer)

Response: Die von Ihnen ausgewählten Fondsarten sind:

1. Standard Life Fonds
2. Managed Portfolios
3. Publikumsfonds verschiedener Fondsgesellschaften

Sources: ['/Users/anuragkotiyal/Desktop/engaige/docs/ParkAllee.pdf:19:7', '/Users/anuragkotiyal/Desktop/engaige/docs/Basispaket+WeitBlick.pdf:16:0', '/Users/anuragkotiyal/Desktop/engaige/docs/Basispaket+WeitBlick.pdf:16:3']


In [21]:
# question 2 in German from query.txt
query = "Wie sieht es mit den Abzügen von der Kirchensteuer aus?"
answer = get_answer(question = query)

In [22]:
print(answer)

Response: Nach dem oben genannten Kontext erfolgt die Kirchensteuerabgabe ab dem 1. Januar 2015 automatisch bei der Abgeltungsteuer, also bei Kapitalerträgen, die als Abgeltung besteuert werden.

Sources: ['/Users/anuragkotiyal/Desktop/engaige/docs/ParkAllee.pdf:7:4', '/Users/anuragkotiyal/Desktop/engaige/docs/Basispaket+WeitBlick.pdf:6:3', '/Users/anuragkotiyal/Desktop/engaige/docs/ParkAllee.pdf:7:6']


In [23]:
# question 3 in German from query.txt
query = "An wen kann ich mich wenden, wenn ich Fragen habe?"
answer = get_answer(question = query)

In [24]:
print(answer)

Response: Sie sollten sich als erstes an Ihren Vermittler wenden. Unsere Servicemitarbeiter sind von Montag bis Freitag von 9:00 bis 17:00 Uhr für Sie da:

* Tel.: 0800 2214747 (kostenfrei)
* Fax: 0800 5892821
* E-Mail: kundenservice@standardlife.de

Sources: ['/Users/anuragkotiyal/Desktop/engaige/docs/ParkAllee.pdf:4:0', '/Users/anuragkotiyal/Desktop/engaige/docs/Basispaket+WeitBlick.pdf:3:0', '/Users/anuragkotiyal/Desktop/engaige/docs/ParkAllee.pdf:31:5']


In [25]:
# question 4 in German from query.txt
query = "Wie bestimmen Sie die Höhe der Auszahlung?"
answer = get_answer(question = query)

In [26]:
print(answer)

Response: Laut dem vorgelegten Text ermitteln wir den Geldwert des Fondsvermögens in Ihrem Vertrag zum maßgeblichen Stichtag. Dieses Fondsvermögen ergibt sich aus der Summe aller Anteilseinheiten der Fonds in Ihrem Vertrag multipliziert mit dem jeweiligen Anteilspreis des Fonds zum Stichtag.

Sources: ['/Users/anuragkotiyal/Desktop/engaige/docs/Basispaket+WeitBlick.pdf:10:9', '/Users/anuragkotiyal/Desktop/engaige/docs/ParkAllee.pdf:12:9', '/Users/anuragkotiyal/Desktop/engaige/docs/Basispaket+WeitBlick.pdf:21:0']


In [27]:
# question 5 in German from query.txt
query = "Welche allgemeinen Versicherungsbedingungen gelten?"
answer = get_answer(question = query)

In [28]:
print(answer)

Response: Nach dem obigen Text gilt Folgendes:

Die Allgemeine Versicherungsbedingungen sind in der Dokumentation "BASIS_PACK_WBWB/D/1006/XIII/03/22" zu finden, insbesondere im Abschnitt "1. Allgemeine Versicherungsbedingungen .....................................................................................................1".

Sources: ['/Users/anuragkotiyal/Desktop/engaige/docs/Basispaket+WeitBlick.pdf:1:0', '/Users/anuragkotiyal/Desktop/engaige/docs/Basispaket+WeitBlick.pdf:10:0', '/Users/anuragkotiyal/Desktop/engaige/docs/Basispaket+WeitBlick.pdf:33:1']
